In [ ]:
#파이스

In [8]:
!pip install python-dotenv
!pip install -U langchain-community
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00


In [3]:
# 기존 설치 제거 + 캐시 클린
!pip uninstall -y faiss-gpu faiss-cpu > /dev/null 2>&1
!rm -rf /root/.cache/pip

In [4]:
# NVIDIA 공식 채널을 통한 CUDA 12.4 설치
!pip install faiss-gpu-cu12 --extra-index-url=https://pypi.nvidia.com --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 67.2 MB/s eta 0:00:00


In [5]:
import torch
print(f"CUDA Version: {torch.version.cuda}")
print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

CUDA Version: 12.4
GPU 사용 가능 여부: True


In [6]:
import faiss
import torch

print(f"[CUDA 버전] {torch.version.cuda}")  # 12.4 출력 확인
print(f"[FAISS GPU 개수] {faiss.get_num_gpus()}")  # 1 이상 출력 확인
print(f"[컴파일 옵션] {faiss.GpuMultipleClonerOptions()}")  # shard=True 확인

[CUDA 버전] 12.4
[FAISS GPU 개수] 1
[컴파일 옵션] <faiss.swigfaiss.GpuMultipleClonerOptions; proxy of <Swig Object of type 'faiss::gpu::GpuMultipleClonerOptions *' at 0x78c2d817fae0> >


In [9]:
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from IPython.display import JSON

In [16]:
text = """2025년 중국의 생성형 AI 스타트업 DeepSeek이 혁신적 기술력으로 글로벌 AI 시장을 뒤흔들고 있습니다.
 671B 매개변수의 V3 모델을 OpenAI 대비 1/10 비용인 단 558만 달러로 개발하며, 2,000개의 NVIDIA H800 칩만을 활용해 초고효율 모델을 선보였습니다.
 R1 추론 모델과 V3 기반 챗봇의 소스코드를 공개하는 오픈소스 전략으로 개발 생태계를 확장하고 있으며, AIME, SWE-bench 등 7개 벤치마크에서 GPT-4와 Claude 3.5와 동등한 성능을 입증했습니다.
 이러한 성과로 공개 3주 만에 미국 iOS 앱스토어 1위를 달성하며 NVIDIA 주가 18% 급락을 유발하는 등 시장에 큰 영향을 미쳤습니다.
 DeepSeek의 기술 혁신은 MoE(Mixture of Experts) 구조에 멀티헤드 잠재 어텐션을 적용해 토큰당 37B 매개변수만 활성화하는 데 성공했습니다.
 다만, 미국 기업들로부터 기술 유출 의혹이 제기되어 알고리즘 효율성에 대한 검증 과정이 진행 중입니다.
"""
#CharacterTextSplitter을 이용한 청킹
splitter = CharacterTextSplitter(
separator="\n",
chunk_size=300,
chunk_overlap=50,
length_function=len
)

#주어진 문장 청킹
chunks = splitter.split_text(text)
print(chunks)

['2025년 중국의 생성형 AI 스타트업 DeepSeek이 혁신적 기술력으로 글로벌 AI 시장을 뒤흔들고 있습니다.\n 671B 매개변수의 V3 모델을 OpenAI 대비 1/10 비용인 단 558만 달러로 개발하며, 2,000개의 NVIDIA H800 칩만을 활용해 초고효율 모델을 선보였습니다. \n R1 추론 모델과 V3 기반 챗봇의 소스코드를 공개하는 오픈소스 전략으로 개발 생태계를 확장하고 있으며, AIME, SWE-bench 등 7개 벤치마크에서 GPT-4와 Claude 3.5와 동등한 성능을 입증했습니다.', '이러한 성과로 공개 3주 만에 미국 iOS 앱스토어 1위를 달성하며 NVIDIA 주가 18% 급락을 유발하는 등 시장에 큰 영향을 미쳤습니다. \n DeepSeek의 기술 혁신은 MoE(Mixture of Experts) 구조에 멀티헤드 잠재 어텐션을 적용해 토큰당 37B 매개변수만 활성화하는 데 성공했습니다. \n 다만, 미국 기업들로부터 기술 유출 의혹이 제기되어 알고리즘 효율성에 대한 검증 과정이 진행 중입니다.']


In [17]:
load_dotenv()
api_key="sk-"
#임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)

#텍스트 임베딩을 FAISS FAISS(Vector Store)에 저장
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [18]:
print(knowledge_base)

In [19]:
question = "deepseek이란?"
references = knowledge_base.similarity_search(question)

In [20]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)


chain = load_qa_chain(llm, chain_type="stuff") #Q&A Interface, 하지만 모든 document를 가져온다
with get_openai_callback() as cb:
    response = chain.run(input_documents=references, question=question)
    print(cb)

Tokens Used: 728
	Prompt Tokens: 503
		Prompt Tokens Cached: 0
	Completion Tokens: 225
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0012045


In [21]:
print(response)

DeepSeek은 중국의 생성형 AI 스타트업 기업으로, 혁신적인 기술력을 바탕으로 글로벌 AI 시장에서 주목을 받고 있습니다. DeepSeek은 MoE(Mixture of Experts) 구조와 멀티헤드 잠재 어텐션을 활용하여 효율적인 모델을 개발하고, 매개변수 활성화에 있어서도 성과를 거두었습니다. 이 회사는 챗봇 및 추론 모델 등의 소스코드를 오픈소스로 공개하여 개발 생태계를 확장하고 있으며, GPT-4와 Claude 3.5와 동등한 성능을 보여주는 등 고성능을 자랑합니다.
